# Práctica final. Fundamentos de tratamiento de datos para Data Science

 Adrián Garrudo Antona

## Ejercicio 1. Análisis de sentimientos de tweets
Ejercicio en el que procesamos dos archivos, uno que contiene tweets y otro que contiene palabras puntuadas según el positivismo de su significado. El objetivo es puntuar cada tweet sumando el valor de los sentimientos de cada palabra que contiene.

## 1.1 Librerías

In [18]:
import pandas as pd
import re
import json 

## 1.2 Carga y procesado de tweets

Dentro del documento tweets hay mucha información de cada publicación, para acceder a el texto debemos cargar el txt y procesarlo como un json dentro de una lista, la lista "tweets" contiene cada línea del texto. Luego generamo filtramos la información ya que la variable "delete" que se encuentra dentro del json no nos interesa. Por último recorremos la lista filtrada y lo convertimos en un dataframe con el que trabajaremos de forma más cómoda. Se muestra en pantalla el resultado de los 5 primeros tweets.

In [19]:
txt = open('tweets.txt')
    
tweets_ = []

for line in txt :
    tweets_.append(json.loads(line))
    
contenido = []

bruto = [ tweet for tweet in tweets_ if 'delete' not in tweet ]
    
for linea_ in bruto:
    contenido.append(linea_['text'])
tweets=pd.DataFrame(contenido)

tweets.columns=['tweets']

tweets.head(5)

,tweets
0,@Brenamae_ I WHALE SLAP YOUR FIN AND TELL YOU ...
1,Metin Şentürk Twitterda @metinsenturk MUHTEŞEM...
2,RT @byunghns: 😭 I LOVE #틴탑 SO MUCH #쉽지않아 IS GO...
3,que hdp maicon lo que le hizo a david luiz jaj...
4,ドライ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！


Procesamos los tweets para poder trtabajar con ellos sin que nos produzcan errores. Las modificaciones pasan por convertir todo a minúsculas, también eliminamos los signos de puntuación para que al tokenizar no lo procese como un token, también eliminamos números ya que buscamos palabras que impliquen sentimientos y eliminamos los dobles espacios en blanco

In [12]:
def procesar_tweet(tweet):

    proceso = tweet.lower()
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    proceso = re.sub(regex , ' ', proceso)
    proceso = re.sub("\d+", ' ', proceso)
    proceso = re.sub("\\s+", ' ', proceso)
    proceso = proceso.split(sep = ' ')
    
    return(proceso)

tweets['texto_tokenizado'] = tweets['tweets'].apply(lambda x: procesar_tweet(x))
tweets[['tweets', 'texto_tokenizado']].head()

,tweets,texto_tokenizado
0,@Brenamae_ I WHALE SLAP YOUR FIN AND TELL YOU ...,"[, brenamae, i, whale, slap, your, fin, and, t..."
1,Metin Şentürk Twitterda @metinsenturk MUHTEŞEM...,"[metin, şentürk, twitterda, metinsenturk, muht..."
2,RT @byunghns: 😭 I LOVE #틴탑 SO MUCH #쉽지않아 IS GO...,"[rt, byunghns, 😭, i, love, 틴탑, so, much, 쉽지않아,..."
3,que hdp maicon lo que le hizo a david luiz jaj...,"[que, hdp, maicon, lo, que, le, hizo, a, david..."
4,ドライ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！,[ドライ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！]


## 1.3 Carga de sentimientos

In [5]:
sentimientost= pd.read_table('sentimientos.txt')
sentimientos=pd.DataFrame(sentimientost)
sentimientos.columns=["palabra","puntos"]
sentimientos.head()

,palabra,puntos
0,abandoned,-2
1,abandons,-2
2,abducted,-2
3,abduction,-2
4,abductions,-2


## 1.4 Dataframe final

Separamos cada tweet en palabras, cada palabra es una fila del dataframe.
También hacemos un join para unir el dataframe de los tweets y el de los sentimientos, de esta manera tendremos el dataframe final "tweets_final".
Agrupamamos el dataframe por tweets y sumamos los puntos de las distintas palabras que pertenecen a cada tweet.
Por último, eliminamos los tweets que tengan una puntuación de cero puntos.

In [6]:
tweets_tok = tweets.explode(column='texto_tokenizado')
tweets_left = pd.merge(
                            left     = tweets_tok,
                            right    = sentimientos,
                            left_on  = "texto_tokenizado", 
                            right_on = "palabra",
                            how      = "left"
                      )

tweets_final = tweets_left[["tweets","texto_tokenizado", "palabra", "puntos"]] \
                      .groupby(["tweets"])\
                      .sum('puntos').reset_index()

                
tweets_final= tweets_final[tweets_final['puntos']!=0]
tweets_final.head()


,tweets,puntos
8,#RT If you Like.... I'm Online Now! http://t.c...,2.0
13,-\nQUEIMAR A LINGUA NO CAFÉ &amp; UMA DAS PIOR...,-1.0
22,"@Aurinth I think it's possible, and sometimes ...",2.0
31,@JessicaFappit and the winner of the best lunc...,7.0
32,@Mugsy4Fly LOL! But she's a famous twitter per...,5.0


## 1.5 Respuesta final ejercicio 1

In [7]:
respuesta=[]
for i in tweets_final.index: 
    print("ESTE TWEET: " + tweets_final["tweets"][i] + " TIENE UN SENTIMIENTO TOTAL DE "+str(tweets_final["puntos"][i])+str(" PUNTOS"))

ESTE TWEET: #RT If you Like.... I'm Online Now! http://t.co/kHSChu3jzz http://t.co/5LLtvleFU6 TIENE UN SENTIMIENTO TOTAL DE 2.0 PUNTOS
ESTE TWEET: -
QUEIMAR A LINGUA NO CAFÉ &amp; UMA DAS PIORES COISAS :( :'( :( TIENE UN SENTIMIENTO TOTAL DE -1.0 PUNTOS
ESTE TWEET: @Aurinth I think it's possible, and sometimes scenes like that can work as long as they're bookended by some sort of development. TIENE UN SENTIMIENTO TOTAL DE 2.0 PUNTOS
ESTE TWEET: @JessicaFappit and the winner of the best lunchtime.visual goes to,,,,,,, TIENE UN SENTIMIENTO TOTAL DE 7.0 PUNTOS
ESTE TWEET: @Mugsy4Fly LOL! But she's a famous twitter personality!!! (It was over Sarah Palin,who I really like - but don't think should run for POTUS) TIENE UN SENTIMIENTO TOTAL DE 5.0 PUNTOS
ESTE TWEET: @Pplschamp2310 At the very least. Her body count has certainly caused the most problems for the club. #SonsOfAnarchy TIENE UN SENTIMIENTO TOTAL DE -2.0 PUNTOS
ESTE TWEET: @RenoiDuVillage lol tavais grail a la cantine d'où t'étais 

## Ejercicio 2
Este problema es una continuación del anterior, vamos a asociar el valor de cada tweet a las palabras que no son sentimientos definidos por el archivo "sentimientos"

## 2.1 Volvemos a procesar el dataframe para ajustarlo al ejercicio

Vamos a darle a las palabras que no estan en "sentimientos" la puntuación que tenga el tweet completo.

In [13]:
tweets_final['texto_tokenizado'] = tweets_final['tweets'].apply(lambda x: procesar_tweet(x))
tweets_final[['tweets', 'texto_tokenizado']].head()
tweets_final_tok = tweets_final.explode(column='texto_tokenizado')
tweets_final_tok.rename(columns={'texto_tokenizado': 'palabra'},
                        inplace=True)

tweets_final_tok = tweets_final_tok.drop(['tweets'], axis=1)
tweets_final_tok.duplicated()
tweets_final_tok = tweets_final_tok.drop_duplicates()


## 2.2 Coincidencias entre dataframe

Realizamos un bucle anidado en el que recorremos los elementos concretos de cada dataframe, buscamos coincidencias entre los sentimientos y las palabras que han dado el valor a los tweets, por último, borramos las palabras que estén en sentimientos 

In [15]:
for i in sentimientos['palabra']:
   for j in tweets_final_tok['palabra']:
       if(i==j):
           tweets_final_tok.drop(tweets_final_tok.loc[tweets_final_tok['palabra']== i].index, inplace=True)

## Respuesta final ejercicio 2

In [16]:
for indice, fila in tweets_final_tok.iterrows(): 
     print("LA PALABRA ", fila['palabra'], "TIENE", fila['puntos'], "PUNTOS")


LA PALABRA  aurinth TIENE 2.0 PUNTOS
LA PALABRA  think TIENE 2.0 PUNTOS
LA PALABRA  it TIENE 2.0 PUNTOS
LA PALABRA  s TIENE 2.0 PUNTOS
LA PALABRA  possible TIENE 2.0 PUNTOS
LA PALABRA  and TIENE 2.0 PUNTOS
LA PALABRA  sometimes TIENE 2.0 PUNTOS
LA PALABRA  scenes TIENE 2.0 PUNTOS
LA PALABRA  that TIENE 2.0 PUNTOS
LA PALABRA  can TIENE 2.0 PUNTOS
LA PALABRA  work TIENE 2.0 PUNTOS
LA PALABRA  as TIENE 2.0 PUNTOS
LA PALABRA  long TIENE 2.0 PUNTOS
LA PALABRA  they TIENE 2.0 PUNTOS
LA PALABRA  re TIENE 2.0 PUNTOS
LA PALABRA  bookended TIENE 2.0 PUNTOS
LA PALABRA  by TIENE 2.0 PUNTOS
LA PALABRA  some TIENE 2.0 PUNTOS
LA PALABRA  sort TIENE 2.0 PUNTOS
LA PALABRA  of TIENE 2.0 PUNTOS
LA PALABRA  development TIENE 2.0 PUNTOS
LA PALABRA  is TIENE 3.0 PUNTOS
LA PALABRA  this TIENE 3.0 PUNTOS
LA PALABRA  not TIENE 3.0 PUNTOS
LA PALABRA  we TIENE 3.0 PUNTOS
LA PALABRA  ve TIENE 3.0 PUNTOS
LA PALABRA  of TIENE 3.0 PUNTOS
LA PALABRA  yet TIENE 3.0 PUNTOS
LA PALABRA  dominos TIENE 3.0 PUNTOS
LA PALABR